### Lösungsvorschlag für Aufgabe 3

#### Vorbereitungen

In [1]:
using LinearAlgebra
using BenchmarkTools
using Test

m, n = (1000,1234)
A = randn(m,m); lu!(A); # Trick für schöne Testmatrizen
B = randn(m,n);

#### Das zu analysierende Programm

In [2]:
function wastutdas!(A,B)
    m,n = size(B)
    for i=n:-1:1
        for j=m:-1:1
            B[j,i] /= A[j,j]
            for k=1:j-1
                B[k,i] -= B[j,i]*A[k,j]
            end
        end
    end
end;

#### Die Analyse: Schritt 1

Da 

   * $i$ nur als Spaltenindex von $B$ auftaucht und 
   * die Spalten von $B$ auch nur über dieses $i$ angesprochen werden und
   * die Reihenfolge der $i$-Durchlaufs egal ist,
   
können wir die äußere `for`-Schleife durch ein `:` für alle Spalten ersetzen:

In [3]:
function wastutdas2!(A,B)
    m,n = size(B)
    # for i=n:-1:1
        for j=m:-1:1
            B[j,:] /= A[j,j]
            for k=1:j-1
                B[k,:] -= B[j,:]*A[k,j]
            end
        end
    # end
end;

#### Die Analyse: Schritt 2

Die innere `for`-Schleife über $k$ können wir  als `1:j` im Zeilenindex „vektorisieren“: $k$ hat nämlich stets die Rolle eines Zeilenindex. Beim Produkt `B[j,:]*A[k,j]` müssen wir mit der Reihenfolge der Multiplikation aufpassen:
Die $j$-te Zeile von $B$ wird mit `A[k,j]` multipliziert und von der $k$-ten Zeile abgezogen:

In [4]:
function wastutdas3!(A,B)
    m,n = size(B)
    # for i=n:-1:1
        for j=m:-1:1
            B[j,:] /= A[j,j]
            # for k=1:j-1
            B[1:j-1,:] -= A[1:j-1,j]*B[[j],:]  # B[[j],:] ist eine weitere Variante für den j-ten Zeilenvektor - nicht ganz so schnell wie conj(B[j,:]')
            # end
        end
    # end
end;

Kleiner Test, ob noch alles passt:

In [5]:
B₀=copy(B); B₃=copy(B);
wastutdas!(A,B₀);
wastutdas3!(A,B₃);
@test isapprox(B₀, B₃)

Test Passed

----------
Zwischenergebnis:

In [6]:
function wastutdas4!(A,B)
    m,n = size(B)
    for j=m:-1:1
        B[j,:] /= A[j,j]
        B[1:j-1,:] -= A[1:j-1,j]*B[[j],:]
    end
end;

#### Die Analyse: Schritt 3

Es geht hier nur die obere Dreiecksmatrix von $A$ ein. Die Umkehrung lautet:

In [7]:
if false
    # Nur zu Doku-Zwecken
    for j=1:m
        X[1:j-1,:] += A[1:j-1,j]*X[[j],:]
        X[j,:] *= A[j,j]
    end
end

Das ist `B=R*X` für `R=UpperTriangular(A)`.
Folglich ist das ursprüngliche Programm eine Form der spaltenweisen Rückwärtssubstitution von $RX=B$.

In [8]:
function wastutdas5!(A,B)
    B[:,:] = UpperTriangular(A)\B
end;

In [9]:
B₀=copy(B); B₅=copy(B);
wastutdas!(A,B₀);
wastutdas5!(A,B₅);
@test isapprox(B₀, B₅)

Test Passed

#### Zugehörige BLAS-Routinge

Mit der BLAS-Routine `TRSM`, siehe (3.5),

In [10]:
?BLAS.trsm

```
trsm(side, ul, tA, dA, alpha, A, B)
```

Return the solution to `A*X = alpha*B` or one of the other three variants determined by determined by [`side`](@ref stdlib-blas-side) and [`tA`](@ref stdlib-blas-trans). Only the [`ul`](@ref stdlib-blas-uplo) triangle of `A` is used. [`dA`](@ref stdlib-blas-diag) determines if the diagonal values are read or are assumed to be all ones.


In [11]:
blas_parameter = [
    'L', # side: gegebene Dreiecksmatrix steht im linearen Gleichungssystem links von der unbekannten Matrix
    'U', # ul: relevant ist das obere Dreieck in A
    'N', # tA: keine Adjunktion
    'N', # da: keine Einheitsdiagonale 
    1.0 # α
    ];

In [12]:
X₁ = copy(B)
wastutdas!(A,X₁)

X₂ = copy(B)
BLAS.trsm!(blas_parameter...,A,X₂)

@test isapprox(X₁, X₂)

Test Passed

#### Messung der Laufzeiten

In [13]:
@benchmark wastutdas!(A,X) setup=(X = copy($B))

BenchmarkTools.Trial: 8 samples with 1 evaluation.
 Range (min … max):  571.935 ms … 794.764 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     667.553 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   678.022 ms ±  79.055 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █          █ █    █              █      █            █      █  
  █▁▁▁▁▁▁▁▁▁▁█▁█▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁█ ▁
  572 ms           Histogram: frequency by time          795 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [14]:
@benchmark wastutdas5!(A,X) setup=(X = copy($B))

BenchmarkTools.Trial: 321 samples with 1 evaluation.
 Range (min … max):  11.309 ms … 20.669 ms  ┊ GC (min … max):  0.00% … 23.61%
 Time  (median):     12.917 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   14.122 ms ±  2.536 ms  ┊ GC (mean ± σ):  11.61% ± 13.64%

    ▃█▄                            ▁                           
  ▄▇███▆▆▃▄▄▄▃▂▄▄▃▃▂▃▃▃▁▂▁▂▃▃▁▁▂▃▆▆█▇▅▃▃▃▃▃▂▃▂▁▂▃▃▃▁▂▂▁▃▃▁▃▁▃ ▃
  11.3 ms         Histogram: frequency by time        20.4 ms <

 Memory estimate: 9.41 MiB, allocs estimate: 2.

In [15]:
@benchmark BLAS.trsm!(blas_parameter...,A,X) setup=(X=copy($B))

BenchmarkTools.Trial: 379 samples with 1 evaluation.
 Range (min … max):   8.086 ms … 18.188 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     10.187 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   10.727 ms ±  1.343 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

             ▂▃▄▆█▆▆▂▃▃▁▁▂ ▁ ▂▁                                
  ▄▁▁▁▁▁▁▁▁▇▄█████████████▇█▇███▇▇▇█▇▇▆▆▇▄▇▆▄▄▄▆▆▄▁▆▄▄▁▁▄▆▁▇▆ ▇
  8.09 ms      Histogram: log(frequency) by time      15.3 ms <

 Memory estimate: 32 bytes, allocs estimate: 1.

Unsere Analsyse hat uns nicht nur dazu gebracht, das ursprüngliche Programm mathematisch zu _verstehen_ und viel _kürzer_ zu formulieren, wir haben es gleich auch noch um etwa einen Faktor 50 für die gegebenen Daten _beschleunigt_.